In [2]:
import numpy as np
import pandas as pd
import datetime as dt
from datetime import timedelta

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from flask import Flask, jsonify

In [3]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [4]:
Base = automap_base()
Base.prepare(engine, reflect=True)

In [5]:
measurement = Base.classes.measurement
station = Base.classes.station

In [6]:
session = Session(engine)

In [7]:
app = Flask(__name__)

In [8]:
last_twelve_months = '2016-08-23'

In [9]:
@app.route("/")
def welcome():
    return (
        f"<p>Welcome! This is the Hawaii weather API. Use of these link will yield JSON results</p>"
        f"/api/v1.0/precipitation<br/>Returns a list of precipitation data for the dates between 8/23/16 and 8/23/17<br/><br/>"
        f"/api/v1.0/stations<br/>Returns a list of the weather stations<br/><br/>"
        f"/api/v1.0/tobs<br/>Returns a list of the Temperature Observations (tobs) for each station for the dates between 8/23/16 and 8/23/17<br/><br/>"
        f"/api/v1.0/start_date/end_date<br/>Returns a list of the minimum temperature, the average temperature, and the max temperature for the dates between the given start date and end date<br/><br/>."
    )

In [10]:
@app.route("/api/v1.0/precipitation")
def precipitation():
    
    prcp_results = session.query(Measurement.date, func.avg(Measurement.prcp)).filter(Measurement.date > last_twelve_months).group_by(Measurement.date).all()
    return jsonify(prcp_results)

In [11]:
@app.route("/api/v1.0/stations")
def stations():
    station_results = session.query(Station.station, Station.name).all()
    return jsonify(station_results)

In [12]:
@app.route("/api/v1.0/tobs")
def tobs():
    tobs_results = session.query(Measurement.date, Measurement.station, Measurement.tobs).filter(Measurement.date > last_twelve_months).all()
    return jsonify(tobs_results)

In [13]:
@app.route("/api/v1.0/<start>/<end>")
def startDateEndDate(start,end):
    range_temp_results = session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).filter(Measurement.date >= start).filter(Measurement.date <= end).all()
    return jsonify(range_temp_results)


In [15]:
if __name__ == "__main__":
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1